# Save and Restore Model
이번 튜토리얼에서는 variable들을 저장하고 다시 불러와서 학습을 재개해도록 하겠습니다. 아래 코드들은 week1 feed_forward_neural_network때와 거의 동일합니다. 마지막 아래 두 셀만 보시면 될 것 같습니다. 자세한 내용은 강의 시간에 설명하겠습니다.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

mnist = input_data.read_data_sets("./mnist", one_hot=True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

print "x_train: ", x_train.shape
print "y_train: ", y_train.shape
print "x_test: ", x_test.shape
print "y_test: ", y_test.shape

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
x_train:  (55000, 784)
y_train:  (55000, 10)
x_test:  (10000, 784)
y_test:  (10000, 10)


In [2]:
def fully_connected(x, dim_in, dim_out, name):
    with tf.variable_scope(name) as scope:
        # create variables
        w = tf.get_variable('w', shape=[dim_in, dim_out], 
                            initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1))
        b = tf.get_variable('b', shape=[dim_out])
        
        # create operations
        out = tf.matmul(x, w) + b
        
        return out   

In [3]:
# Create model
def neural_network(x, dim_in=784, dim_h=500, dim_out=10):
    # 1st hidden layer with ReLU
    h1 = fully_connected(x, dim_in, dim_h, name='h1')
    h1 = tf.nn.relu(h1)
    
    # 2nd hidden layer with ReLU
    h2 = fully_connected(h1, dim_h, dim_h, name='h2')
    h2 = tf.nn.relu(h2)
    
    # output layer with linear
    out = fully_connected(h2, dim_h, dim_out, name='out')
    
    return out

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [5]:
# Construct model with default value
out = neural_network(x)

In [6]:
# loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001).minimize(loss)

# Test model
pred = tf.argmax(out, 1)
target = tf.argmax(y, 1)
correct_pred = tf.equal(pred, target)

# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### 매 epoch마다 모든 variable들을 save합니다.

In [7]:
batch_size = 100
save_every = 1

if not os.path.exists('model/'):
    os.makedirs('model/')

# launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    # initialize tensor variables
    tf.initialize_all_variables().run()
    saver = tf.train.Saver(max_to_keep=20)
    # training cycle
    for epoch in range(1):
        avg_loss = 0.
        n_iters_per_epoch = int(mnist.train.num_examples / batch_size)
        # loop over all batches
        for i in range(n_iters_per_epoch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            # run optimization op (backprop) and loss op (to get loss value)
            _, c = sess.run([optimizer, loss], feed_dict={x: x_batch, y: y_batch})
            # compute average loss
            avg_loss += c / n_iters_per_epoch
        print "Epoch %d, Loss: %.3f"% (epoch+1, avg_loss)
        if epoch % save_every == 0:
            saver.save(sess, save_path='model/fc', global_step=epoch+1)
            
    print "Finished training!"
    print "\nTest accuracy:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})

Epoch 1, Loss: 0.985
Finished training!

Test accuracy: 0.9065


### Save한 variable들을 다시 불러와 train을 재개합니다.

In [8]:
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    saver.restore(sess, 'model/fc-1')
    # training cycle
    for epoch in range(3):
        avg_loss = 0.
        n_iters_per_epoch = int(mnist.train.num_examples / batch_size)
        # loop over all batches
        for i in range(n_iters_per_epoch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            # run optimization op (backprop) and loss op (to get loss value)
            _, c = sess.run([optimizer, loss], feed_dict={x: x_batch, y: y_batch})
            # compute average loss
            avg_loss += c / n_iters_per_epoch
        print "Epoch %d, Loss: %.3f"% (epoch+1, avg_loss)
        if epoch % save_every == 0:
            saver.save(sess, save_path='model/fc', global_step=epoch+1)
            
    print "Finished training!"
    print "\nTest accuracy:", sess.run(accuracy, {x: mnist.test.images, y: mnist.test.labels})

Epoch 1, Loss: 0.264
Epoch 2, Loss: 0.195
Epoch 3, Loss: 0.155
Finished training!

Test accuracy: 0.9563
